# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v6.2.0 (Open Box Edition)
**Güncelleme Notları:**
- **🎛️ Açık Hiperparametreler:** Optuna'nın kullandığı tüm ayarlar (Öğrenme hızı, derinlik vb.) artık bu sayfada, gözünün önünde. İstediğin gibi değiştirip deneyebilirsin.
- **Tam Kontrol:** Her şeyi sen yönetiyorsun.
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap (Temiz Kurulum)
import os
import shutil

# 1. Kök dizine dön
try:
    os.chdir('/content')
except:
    pass

# 2. Varsa eski projeyi SİL (Her şeyi sıfırla ki eski hatalar kalmasın)
if os.path.exists('newproje'):
    print("Eski proje dosyaları temizleniyor...")
    shutil.rmtree('newproje')

# 3. Sıfırdan İndir
print("Projeyi GitHub'dan sıfırdan çekiyorum...")
!git clone https://github.com/onndd/newproje.git

# 4. Çalışma dizinine gir
if os.path.exists('newproje/avci'):
    os.chdir('newproje/avci')
    print(f"✅ Kurulum Başarılı. Çalışma Dizini: {os.getcwd()}")
    
    # --- VERİTABANI KURTARMA ---
    drive_db_path = '/content/drive/MyDrive/newproje/avci/jetx.db'
    local_db_path = 'jetx.db'
    
    if not os.path.exists('/content/drive'):
        try:
            from google.colab import drive
            drive.mount('/content/drive')
        except:
            pass

    if os.path.exists(drive_db_path):
        print(f"🔄 Gerçek Veritabanı Drive'dan kopyalanıyor... ({os.path.getsize(drive_db_path)/1024/1024:.2f} MB)")
        shutil.copy(drive_db_path, local_db_path)
        print("✅ Veritabanı güncellendi.")
    else:
        print("⚠️ UYARI: Drive'da 'jetx.db' bulunamadı! Veritabanı boş olabilir.")
else:
    print("❌ HATA: İndirme başarısız oldu!")

print("GPU Durumu:")
!nvidia-smi

# Kütüphane Kurulumları
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("Kurulum ve Hazırlık Tamamlandı! Artık modüller bulunabilir.")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

try:
    # Modülleri içe aktar (Absolute Import)
    import train_avci
    import models_avci
    import model_ae_avci
    import model_hmm_avci
    import lightgbm as lgb
    import numpy as np
    from sklearn.metrics import confusion_matrix
    
    # Fonksiyonları kısayol olarak al
    load_and_prep = train_avci.load_and_prep
    visualize_performance = train_avci.visualize_performance
    get_scoring_params = train_avci.get_scoring_params
    
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Modül import hatası. Detay: {e}")

if 'load_and_prep' in locals():
    try:
        df = load_and_prep(limit=100000)
        print(f"Veri Hazır: {len(df)} satır. (Sütunlar: {len(df.columns)})")
    except Exception as e:
         print(f"❌ VERİ YÜKLEME HATASI: {e}")

## 🎛️ Optuna Hiperparametre Ayarları (BEYİN)
Buradaki değerleri değiştirerek yapay zekanın arama alanını yönetebilirsin.

In [ ]:
def run_optuna_for_target(target, epochs=30):
    """
    Belirtilen hedef için Optuna optimizasyonunu Notebook içinde çalıştırır.
    Böylece parametreleri GÖREBİLİRSİN.
    """
    print(f"\n🚀 {target}x Hedefi İçin Eğitim Başlıyor...")
    
    # 1. Hedef Sütunu Hazırla
    y_col = f'target_{str(target).replace(".","_")}'
    if y_col not in df.columns:
        print(f"❌ Hata: {y_col} sütunu bulunamadı!")
        return
        
    y_train = df[y_col].iloc[:split_idx]
    y_val = df[y_col].iloc[split_idx:]
    scoring = get_scoring_params(target)

    # 2. OPTUNA OBJECTIVE FONKSİYONU (Parametreler Burada)
    def objective(trial):
        params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            
            # --- DÜZENLENEBİLİR ALAN ---
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),     # Öğrenme Hızı
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),              # Ağaç Karmaşıklığı
            'max_depth': trial.suggest_int('max_depth', 3, 12),                  # Derinlik
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0), # Özellik Kullanımı
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0), # Veri Kullanımı
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            # ---------------------------
            
            'device': 'gpu',
            'gpu_platform_id': 0,
            'gpu_device_id': 0
        }
        
        # GPU Fallback
        try:
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val)
            model = lgb.train(params, train_data, valid_sets=[val_data], callbacks=[lgb.early_stopping(30)])
        except Exception:
            params['device'] = 'cpu'
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val)
            model = lgb.train(params, train_data, valid_sets=[val_data], callbacks=[lgb.early_stopping(30)])

        preds = model.predict(X_val)
        
        # Skorlama (Profit Max)
        best_score = -float('inf')
        thresholds = np.arange(0.50, 0.99, 0.01)
        for thr in thresholds:
            pred_labels = (preds > thr).astype(int)
            tn, fp, fn, tp = confusion_matrix(y_val, pred_labels).ravel()
            score = (tp * scoring['TP']) + (tn * scoring['TN']) - (fp * scoring['FP']) - (fn * scoring['FN'])
            if score > best_score: best_score = score
        return best_score

    # 3. Optuna'yı Çalıştır
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=epochs)
    
    print(f"En İyi Parametreler: {study.best_params}")
    
    # 4. Final Model ve Görselleştirme
    final_params = study.best_params
    final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1, 'device': 'gpu'})
    
    final_model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
    final_model.save_model(f'models/avci_lgbm_{str(target).replace(".","_")}.txt')
    
    visualize_performance(final_model, X_val, y_val, target)
    try: optuna.visualization.plot_optimization_history(study).show()
    except: pass

## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
if 'model_ae_avci' in locals():
    ae = model_ae_avci.train_autoencoder(df)

In [ ]:
if 'model_hmm_avci' in locals():
    hmm_model = model_hmm_avci.train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (LightGBM)
Aşağıdaki hücrelerde her hedef için eğitim yapabilirsin.

In [ ]:
# Veri Setini Hazırla
features = [c for c in df.columns if 'target' not in c and 'result' not in c and 'value' not in c and 'id' not in c]
X = df[features]
split_idx = int(len(df) * 0.85)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]

In [ ]:
# --- 1.50x HEDEFİ ---
run_optuna_for_target(1.5, epochs=30)

In [ ]:
# --- 3.00x HEDEFİ ---
run_optuna_for_target(3.0, epochs=30)

In [ ]:
# --- 5.00x HEDEFİ ---
run_optuna_for_target(5.0, epochs=50)

In [ ]:
# --- 10.0x HEDEFİ ---
run_optuna_for_target(10.0, epochs=50)

In [ ]:
# --- 50.0x HEDEFİ ---
run_optuna_for_target(50.0, epochs=100)

In [ ]:
# --- 100.0x HEDEFİ ---
run_optuna_for_target(100.0, epochs=100)

In [ ]:
# --- 1000.0x HEDEFİ (God Mode) ---
run_optuna_for_target(1000.0, epochs=150)

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Tüm Modelleri Ziple
import os
if not os.path.exists('models'):
    os.makedirs('models')
    
!zip -r avci_models.zip models/

# İndir
try:
    from google.colab import files
    files.download('avci_models.zip')
    print("Dosya indiriliyor... 'avci_models.zip'")
except Exception as e:
    print(f"İndirme hatası: {e}. Sol taraftaki dosya gezgininden 'avci_models.zip' dosyasını sağ tıklayıp indirebilirsin.")

# Varsa Drive'a Kopyala
if os.path.exists('/content/drive/MyDrive'):
    !cp avci_models.zip /content/drive/MyDrive/avci_backup_latest.zip
    print("Yedek Drive'a (Ana Dizin) kopyalandı.")